In [1]:
import pandas as pd 

def get_columns_details(year,type,AC):
    if type == 'AE':
        election_df = pd.read_csv('data/Chhattisgarh_AE_with_hindinames.csv')
    # if type == 'GA':
    #     election_df = pd.read_csv('data/Maharashtra_GA.csv')
    filtered_election_df = election_df[(election_df['Year'] == year) & (election_df['Constituency_No'] == AC) & (~election_df['Candidate'].isin(['NOTA','None of the Above'])) ]
    return  filtered_election_df.shape[0]


In [2]:
import json

def get_form_20_details(year, type):
    if year == 2018 and type == 'AE':
        file_path = 'results/name_mappings/chhattisgarh_2018_AE.json'
    # elif year == 2019 and type == 'GA':
    #     file_path = 'results/name_mappings/maharastra_2019_GA.json'
        
    with open(file_path, 'r') as file: 
        names_mapping = json.load(file) 
        
    if type == 'AE':
        election_df = pd.read_csv('data/Chhattisgarh_AE.csv')
    # if type == 'GA':
    #     election_df = pd.read_csv('data/Maharashtra_GA.csv')
        
    return names_mapping, election_df

def get_candidate_and_form_20_names(form_20_json, election_df, AC, year, top_3 = False):
    top_3 = [1,2,3]
    if top_3: 
        # Define conditions
        condition_general = (
            (election_df['Year'] == year) &
            (election_df['Constituency_No'] == AC) &
            (~election_df['Candidate'].isin(['None of the Above', 'NOTA']))
        )

        condition_inc = (
            (election_df['Party'] == 'INC')
        )
        
        # Use bitwise OR to combine conditions
        filtered_election_df = election_df[
            (condition_general & election_df['Position'].isin(top_3)) |
            (condition_inc & condition_general)
        ]
    else: 
        filtered_election_df = election_df[(election_df['Year'] == year) & (election_df['Constituency_No'] == AC) & (election_df['Candidate'] != 'None of the Above')  ]

    candidates_list = filtered_election_df['Candidate'].tolist()
    form_20_names = [i['name']  for i in form_20_json if i['year'] == year and i['AC'] == AC]
    
    return candidates_list, form_20_names



In [3]:
year = 2018
# type = 'GA'
type = 'AE'
name_mapping, election_df  = get_form_20_details(year, type)

FileNotFoundError: [Errno 2] No such file or directory: 'results/name_mappings/chhattisgarh_2018_AE.json'

In [ ]:
import csv
import json
from fuzzywuzzy import fuzz

def fuzzy_match_name_with_tolerance(name, name_list, tolerance=5):
    cleaned_name = name.lower().replace(" ", "").replace(".","")
    max_score = -1
    matched_name = None
    for candidate_name in name_list:
        cleaned_candidate_name = candidate_name.lower().replace(" ", "").replace(".","")
        score = fuzz.ratio(cleaned_name, cleaned_candidate_name)
        len_sum = len(cleaned_name) + len(cleaned_candidate_name)
        levenshtein_distance = len_sum - score * len_sum / 100
        if levenshtein_distance <= tolerance:
            if score >= max_score:
                max_score = score
                matched_name = candidate_name
            
    # If no matches are found, process name parts
    if matched_name is None:
        sub_tolerance = 3
        name = name.replace(".","")
        name_parts = sorted(name.split(), key=len, reverse=True)  # Split and sort by length
        filtered_name_parts = [part for part in name_parts if len(part) > 2]  # Filter parts greater than 3 characters
        for part in filtered_name_parts:
            part_cleaned = part.lower().replace(" ", "").replace(".","")
            for candidate_name in name_list:
                cleaned_candidate_name = candidate_name.lower().replace(" ", "").replace(".","")
                
                best_score = 0
                best_distance = float('inf')

                # Sliding window to check every possible substring of candidate name
                for start in range(len(cleaned_candidate_name) - len(part_cleaned) + 1):
                    end = start + len(part_cleaned)
                    candidate_substring = cleaned_candidate_name[start:end]
                    
                    # Calculate fuzzy score and distance for the substring
                    score = fuzz.ratio(part_cleaned, candidate_substring)
                    len_sum = len(part_cleaned) + len(candidate_substring)
                    levenshtein_distance = len_sum - score * len_sum / 100
                    
                    # Update the best score and distance found so far
                    if score > best_score or (score == best_score and levenshtein_distance < best_distance):
                        best_score = score
                        best_distance = levenshtein_distance
                
                # If the best score and distance within tolerance, accept it as a match
                if best_distance <= sub_tolerance:
                    matched_name = candidate_name
            if matched_name:
                break
        
    return matched_name

In [4]:
def get_column_index(column_name, form_20_json, year, AC):
    for i in form_20_json:
        if i['year'] == year and i['AC'] == AC and i['name'] == column_name:
            return i['column']
    return None

def get_party(election_df,name, year , AC) : 
    party = election_df[(election_df['Year'] == year) & (election_df['Constituency_No'] == AC) & (election_df['Candidate'] == name)]['Party'].values[0]
    return party


In [5]:
def get_party_mappings(year, AC): 
    results = []
    candidates_list, form_20_names = get_candidate_and_form_20_names(name_mapping,election_df, AC,year, top_3=True)
    
    for candidate in candidates_list: 
        matched_name = fuzzy_match_name_with_tolerance(candidate, form_20_names, 5)
        party = get_party(election_df,candidate, year, AC)
        if matched_name: 
            column_index = get_column_index(matched_name, name_mapping, year, AC)
            results.append({
                "column" : column_index,
                "party" : party
            })
    return results

In [6]:
import pandas as pd
import numpy as np

def clean_excel_file(df, year, AC, type):
    num_of_candidates = get_columns_details(year,type,AC)
    party_mappings = get_party_mappings(year, AC)
    
    # Step 1: Rename Columns based on their expected order and content
    num_cols = df.shape[1]
    number_of_other_columns = num_cols - num_of_candidates - 5 
    if number_of_other_columns <= 0: 
        raise Exception("Issue in the number of columns in the excel file", year, AC)
    # core_cols = ['SN', 'Polling_Station']
    core_cols = [f'col_{i}' for i in range(1, number_of_other_columns + 1)]

    # Continue with the remaining expected column names
    candidate_cols = [f'col_{i}' for i in range(num_cols - num_of_candidates - 4, num_cols - 5 + 1)]
    
    core_cols += candidate_cols
    
    nota_check = str(df.columns[num_cols - 5]).lower()  # Lowercase the third column name to check properly
    if 'none' in nota_check:
        core_cols.append('NOTA')
        core_cols += ['Total_Valid_Votes', 'Total_Rejected_Votes', 'Total', 'Total_Votes_Tendered']
    else : 
        core_cols += ['Total_Valid_Votes', 'Total_Rejected_Votes', 'NOTA', 'Total', 'Total_Votes_Tendered']

    # Apply the new column names
    df.columns = core_cols

    df.rename(columns={'col_1': 'SN'}, inplace=True)
    
    if number_of_other_columns > 1: 
        # drop those columns 
        df = df.drop(df.columns[1:number_of_other_columns], axis=1)
    # Step 2: Clean all cells in the dataframe to remove unwanted characters
    df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))

    # Step 3: Replace 'nan' strings with actual NaN values
    df.replace(to_replace='^nan$', value=np.nan, regex=True, inplace=True)

    # # Step 4: Convert applicable columns to numeric types
    # # Define columns to exclude from numeric conversion (text columns)
    # exclude_columns = ['SN', 'Polling_Station']

    # Identify columns that should be processed (all except the excluded ones)
    # columns_to_process = df.columns.difference(exclude_columns)

    # Apply numeric conversion only to the appropriate columns
    df = df.apply(pd.to_numeric, errors='coerce')

    # Step 5: Filter out rows where any cell is non-numeric (where it should be numeric)
    df = df[df.applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]

    df = df.apply(pd.to_numeric, errors='coerce')
    
    
    # Step 1: Identify columns
    col_names = [col for col in df.columns if 'col' in col] + ['NOTA']

    # Step 2: Calculate the sum of the relevant columns
    df['calculated_sum'] = df[col_names].sum(axis=1)

    # Step 3: Calculate the absolute difference between calculated_sum and Total
    df['difference'] = (df['Total'] - df['calculated_sum']).abs()

    # Step 4: Drop rows where the difference is greater than 50
    df = df[df['difference'] <= 75]

    # Dropping the helper columns as they are no longer needed (optional)
    df.drop(columns=['calculated_sum', 'difference'], inplace=True)
    
    for mapping in party_mappings: 
        df = df.rename(columns={f'col_{mapping["column"]}': mapping["party"]})
    df = df[df['SN'].notnull()]

    return df

In [7]:
# import json 
# with open('constituency_mapping.json') as f:
#     constituency_mapping = json.load(f)
    
# constituency_mapping

In [8]:

import os 
import re

def clean_and_dump_excel_files(excel_dir, output_dir, log_file_name, year, type):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(excel_dir):
        if filename.endswith('.xlsx'):
            try: 
                df = pd.read_excel(os.path.join(excel_dir, filename))
                # match = re.search(r'combined_JSON_([a-zA-Z]+)_AssemblyElection_(\d{4})_A(\d{3}).json.xlsx', filename)
                # match = re.search(r'combined_JSON_([a-zA-Z]+)_(\d{4})_AC_(\d{3}).json.xlsx', filename)
                # match = re.search(r'combined_JSON_([a-zA-Z]+)_LokSabha_Election_(\d{4})_AC_(\d{3}).json.xlsx', filename)
                match = re.search(r'combined_JSON_([a-zA-Z]+)_(\d{4})_AC_(\d{2}).json.xlsx',filename)


                AC = None
                if match:
                    AC = int(match.group(3))
                cleaned_df = clean_excel_file(df, year, AC, type)
                cleaned_df['State'] = 'MH'
                cleaned_df['Year'] = year
                cleaned_df['Constituency'] = AC
                output_file_path = os.path.join(output_dir, f"{AC}.xlsx")
                cleaned_df.to_excel(output_file_path, index=False)
                
            except Exception as exc: 
                with open(log_file_name, 'a') as f: 
                    f.write(f"Error processing file: {filename} - {exc}\n")
                continue
        

In [10]:
excel_dir = 'output/Parsed_Excel/Chhattisgarh/AE_2018'
output_dir = 'output/cleaned_election_data/CH/AE_2018'
log_file_name = "logs/chhattisgarh_cleaning_data_2018_AE.txt"
year = 2018
type = 'AE'

clean_and_dump_excel_files(excel_dir, output_dir, log_file_name, year, type)

FileNotFoundError: [Errno 2] No such file or directory: 'output/Parsed_Excel/Chhattisgarh/AE_2018'

In [40]:
# excel_dir = 'results/Parsed_Excel/Maharastra/Assembly Election 2019'
# output_dir = 'results/cleaned_election_data/MH/2019_AE'
# log_file_name = "logs/maharasthra_cleaning_data_2019_AE.txt"
# year = 2019
# type = 'AE'

# excel_dir = 'results/Parsed_Excel/Maharastra/Lok Sabha Election 2019'
# output_dir = 'results/cleaned_election_data/MH/2019_GA'
# log_file_name = "logs/maharasthra_cleaning_data_2019_GA.txt"
# year = 2019
# type = 'GA'

# excel_dir = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined'
# output_dir = 'cleaned_election_data/KA/2018'
# log_file_name = "logs/karnataka_assembly_election_2018_cleaning_log.txt"
# year = 2018

# excel_dir = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined'
# output_dir = 'cleaned_election_data/KA/2014'
# log_file_name = "logs/karnataka_assembly_election_2014_cleaning_log.txt"
# year = 2014

# clean_and_dump_excel_files(excel_dir, output_dir, log_file_name, year, type)


/tmp/ipykernel_288484/3613950028.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_288484/3613950028.py:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df.applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_288484/3613950028.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c in str(x) if c.isalnum() or c == '.'))
/tmp/ipykernel_288484/3613950028.py:53: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df.applymap(lambda x: pd.isna(x) or isinstance(x, (int, float))).all(axis=1)]
/tmp/ipykernel_288484/3613950028.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: ''.join(c for c i

In [18]:
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_28721687260574.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_70041687260593.json.xlsx' #-223_2023
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_26691687260574.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_26691687260574.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_27121687260536.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2023/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2023_72381687260175.json.xlsx'

# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2018_187.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2018_019.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2018_001.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2018_213.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Assembly Election 2018/Excel_Json_combined/combined_JSON_karnataka_AssemblyElection_2018_094.json.xlsx'

# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-199.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-196.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-218.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-180.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-142.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-130.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-094.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-078.json.xlsx'
# file_path = 'Parsed_Excel/Karnataka/Lok Sabha Election 2014/Excel_Json_combined/combined_JSON_karnataka_LokSabha_2014_F20-038.json.xlsx'

# file_path = 'results/Parsed_Excel/Maharastra/Assembly Election 2019/combined_JSON_Maharastra_2019_AC_070.json.xlsx'

file_path = 'results/Parsed_Excel/Maharastra/Lok Sabha Election 2019/combined_JSON_MH_LokSabha_Election_2019_AC_007.json.xlsx'

year = 2019
type = 'GA'
# type = 'AE'
name_mapping, election_df  = get_form_20_details(year, type)
df = pd.read_excel(file_path) 
print("initial_shape>>", df.shape)
cleaned_df = clean_excel_file(df, 2019, 7, 'GA')

print("after shape>>",cleaned_df.shape)

UnboundLocalError: local variable 'file_path' referenced before assignment

In [ ]:
cleaned_df.head(10)